# 02 - Advanced Types

Complex types, nested models, enums, and more.

In [1]:
from typing import Optional, List, Union
from pydantic import BaseModel
from enum import Enum
import polars as pl
from poldantic import to_polars_schema

## 1. Nested Models

In [2]:
class Address(BaseModel):
    street: str
    city: str
    zip: int

class Customer(BaseModel):
    id: int
    name: str
    address: Address

schema = to_polars_schema(Customer)
print('Nested schema:', schema)

# Create DataFrame
data = [{
    'id': 1,
    'name': 'Alice',
    'address': {'street': '123 Main', 'city': 'NYC', 'zip': 10001}
}]

df = pl.DataFrame(data, schema=schema)
print(df)

Nested schema: {'id': Int64, 'name': String, 'address': Struct({'street': String, 'city': String, 'zip': Int64})}
shape: (1, 3)
┌─────┬───────┬──────────────────────────┐
│ id  ┆ name  ┆ address                  │
│ --- ┆ ---   ┆ ---                      │
│ i64 ┆ str   ┆ struct[3]                │
╞═════╪═══════╪══════════════════════════╡
│ 1   ┆ Alice ┆ {"123 Main","NYC",10001} │
└─────┴───────┴──────────────────────────┘


## 2. Enums

In [3]:
class Status(str, Enum):
    PENDING = 'pending'
    ACTIVE = 'active'
    COMPLETED = 'completed'

class Task(BaseModel):
    id: int
    title: str
    status: Status

schema = to_polars_schema(Task)
print('Enum schema:', schema)

tasks = [
    {'id': 1, 'title': 'Setup', 'status': 'completed'},
    {'id': 2, 'title': 'Testing', 'status': 'active'},
]

df = pl.DataFrame(tasks, schema=schema)
print(df)

Enum schema: {'id': Int64, 'title': String, 'status': Enum(categories=['pending', 'active', 'completed'])}
shape: (2, 3)
┌─────┬─────────┬───────────┐
│ id  ┆ title   ┆ status    │
│ --- ┆ ---     ┆ ---       │
│ i64 ┆ str     ┆ enum      │
╞═════╪═════════╪═══════════╡
│ 1   ┆ Setup   ┆ completed │
│ 2   ┆ Testing ┆ active    │
└─────┴─────────┴───────────┘


## 3. Lists and Complex Collections

In [4]:
class Article(BaseModel):
    id: int
    title: str
    tags: List[str]
    scores: List[float]

schema = to_polars_schema(Article)
print('List schema:', schema)

articles = [
    {'id': 1, 'title': 'Intro', 'tags': ['python', 'tutorial'], 'scores': [4.5, 4.8]},
    {'id': 2, 'title': 'Advanced', 'tags': ['python', 'advanced'], 'scores': [4.9, 4.7]},
]

df = pl.DataFrame(articles, schema=schema)
print(df)

List schema: {'id': Int64, 'title': String, 'tags': List(String), 'scores': List(Float64)}
shape: (2, 4)
┌─────┬──────────┬────────────────────────┬────────────┐
│ id  ┆ title    ┆ tags                   ┆ scores     │
│ --- ┆ ---      ┆ ---                    ┆ ---        │
│ i64 ┆ str      ┆ list[str]              ┆ list[f64]  │
╞═════╪══════════╪════════════════════════╪════════════╡
│ 1   ┆ Intro    ┆ ["python", "tutorial"] ┆ [4.5, 4.8] │
│ 2   ┆ Advanced ┆ ["python", "advanced"] ┆ [4.9, 4.7] │
└─────┴──────────┴────────────────────────┴────────────┘


## 4. Deeply Nested Structures

In [5]:
class Item(BaseModel):
    name: str
    quantity: int

class Order(BaseModel):
    order_id: int
    items: List[Item]
    total: float

# Note: List[BaseModel] maps to List(Struct(...))
schema = to_polars_schema(Order)
print('Deeply nested schema:', schema)

Deeply nested schema: {'order_id': Int64, 'items': List(Struct({'name': String, 'quantity': Int64})), 'total': Float64}
